In [1]:
!pip install arxiv

In [7]:
import sys, importlib, os
print("Python exe:", sys.executable)

try:
    import langchain_ollama
    print("langchain_ollama file:", langchain_ollama.__file__)
    try:
        from importlib.metadata import version
    except Exception:
        from importlib_metadata import version  # py<3.8
    print("langchain-ollama version:", version("langchain-ollama"))
except Exception as e:
    print("langchain_ollama not importable:", e)

Python exe: /Users/varunsardana/Desktop/Langchain-Project/Langchain/bin/python
langchain_ollama file: /Users/varunsardana/Desktop/Langchain-Project/Langchain/lib/python3.10/site-packages/langchain_ollama/__init__.py
langchain-ollama version: 0.3.7


In [8]:
import os
os.environ["USER_AGENT"] = "Mozilla/5.0 (Mac) LangChainNotebook/1.0"

In [9]:
# robust import that works across versions
try:
    from langchain_ollama.embeddings import OllamaEmbeddings   # ✅ 0.3.x path
except ImportError:
    from langchain_community.embeddings import OllamaEmbeddings # legacy fallback

In [10]:
emb = OllamaEmbeddings(model="nomic-embed-text")
vec = emb.embed_query("hello world")   # should return a vector (list of floats)
len(vec), type(vec[:3])

(768, list)

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [17]:
api_wrapper= WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki= WikipediaQueryRun(api_wrapper=api_wrapper)


In [18]:
wiki.name

'wikipedia'

In [19]:
result= wiki.invoke({"query": "Who founded Wikipedia?"})
print(result)

Page: Wikipedia
Summary: Wikipedia is a free online encyclopedia written and maintained by a community of volunteers, known as Wikipedians, through open collaboration and the wiki software MediaWiki. 


In [12]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter   

loader = WebBaseLoader("https://en.wikipedia.org/wiki/Artificial_intelligence")
docs = loader.load()
documents= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb= FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text"))
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11878aa70>, search_kwargs={})

In [14]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool= create_retriever_tool(retriever, "artificial_intelligence search", " Search for when you need to find information on artificial intelligence")

In [15]:
retriever_tool.name

'artificial_intelligence search'

In [16]:
# Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper= ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [20]:
tools = [wiki, arxiv, retriever_tool]

In [21]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/varunsardana/Desktop/Langchain-Project/Langchain/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='artificial_intelligence search', description=' Search for when you need to find information on artificial intelligence', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10fe8c5e0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_

In [22]:
from langchain_ollama import ChatOllama
llm= ChatOllama(model= "gemma3:1b", temperature=0)


In [26]:
from langchain import hub

prompt = hub.pull("hwchase17/react")



In [ ]:
# Agents
from langchain.agents import create_react_agent, AgentExecutor

tools = [wiki, arxiv, retriever_tool]

agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [28]:
# Agent Executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools= tools, verbose= True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nartificial_intelligence search(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(tags=[\'FAISS\', \'OllamaEmbeddings\'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11878aa70>

In [29]:
agent_executor.invoke({"input": "Tell me about Artificial Intelligence"})



> Entering new AgentExecutor chain...
Thought: I need to find information about Artificial Intelligence. I’ll start with a general search on Wikipedia.
Action: wikipedia
Action Input: “Artificial intelligence”Page: Artificial intelligence
Summary: Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoninI need to understand the basics of AI. Let’s start with a general overview from Wikipedia.
Action: wikipedia
Action Input: “Artificial intelligence”Page: Artificial intelligence
Summary: Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoninI understand the basics of AI. Let’s delve deeper.
Action: wikipedia
Action Input: “Artificial intelligence”Page: Artificial intelligence
Summary: Artificial intelligence (AI) is the capability of computational systems to perform tasks typic

{'input': 'Tell me about Artificial Intelligence',
 'output': 'The page provides a basic overview of AI, including its definition and key aspects like learning and reasoning. It also mentions different types like machine learning, deep learning, and natural language processing.'}

In [30]:
agent_executor.invoke({"input": "Tell me about Machine Learning and its applications"})



> Entering new AgentExecutor chain...
I need to answer about Machine Learning and its applications. Let’s start with a general overview.
Action: wikipedia
Action Input: “Machine learning”Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genI need to provide a general overview of Machine Learning. It’s a broad field with many sub-disciplines. Let’s start with a basic definition and some key aspects.
Action: wikipedia
Action Input: “Machine learning”Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genI need to provide a general overview of Machine Learning and its applications. Let’s start with a basic definition and some key aspects.
Action: wikipedia
Action Input: “Machine learning”Page: Machin

{'input': 'Tell me about Machine Learning and its applications',
 'output': 'Agent stopped due to iteration limit or time limit.'}